> 使用预训练的BERT模型进行建模的思路步骤如下：
>
> - 数据预处理：首先，对文本数据进行预处理，包括文本清洗（如去除特殊字符、标点符号）、分词等操作。可以使用常见的NLP工具包（如NLTK或spaCy）来辅助进行预处理。
> - 构建训练所需的dataloader与dataset，构建Dataset类时，需要定义三个方法__init__，__getitem__， __len__，其中__init__方法完成类初始化，__getitem__要求返回返回内容和label，__len__方法返回数据长度
> - 构造Dataloader，在其中完成对句子进行编码、填充、组装batch等动作：
> - 定义预测模型利用预训练的BERT模型来解决文本二分类任务，我们将使用BERT模型编码中的[CLS]向量来完成二分类任务[CLS]就是classification的意思，可以理解为用于下游的分类任务。

> 主要用于以下两种任务：

> - 单文本分类任务：对于文本分类任务，BERT模型在文本前插入一个[CLS]符号，并将该符号对应的输出向量作为整篇文本的语义表示，用于文本分类，如下图所示。可以理解为：与文本中已有的其它字/词相比，这个无明显语义信息的符号会更“公平”地融合文本中各个字/词的语义信息。
>
> - 在模型设计中思路就体现为我们取出文本数据经过向量化后的[CLS]向量，然后经过二分类预测层得到最终的结果

In [ ]:
outputs = self.bert(**src).last_hidden_state[:, 0, :]
self.predictor(outputs)
self.predictor = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )


- 模型训练和评估：使用训练集对选定的机器学习模型进行训练，然后使用测试集进行评估。评估指标可以选择准确率、精确率、召回率、F1值等。
- 调参优化：如果模型效果不理想，可以尝试调整特征提取的参数（如词频阈值、词袋大小等）或机器学习模型的参数，以获得更好的性能。
- 在这个进阶实践中，我们使用深度学习方法，一般会遵循以下流程：
  数据收集与准备->模型定义->模型训练->模型评估与优化->结果输出

##### 导入我们本次Baseline代码所需的模块

In [1]:
#import 相关库
#导入前置依赖
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# 用于加载bert模型的分词器
from transformers import AutoTokenizer
# 用于加载bert模型
from transformers import BertModel
from pathlib import Path


##### 设置全局配置

In [2]:
batch_size = 16
# 文本的最大长度
text_max_length = 128
# 总训练的epochs数，我只是随便定义了个数
epochs = 100
# 学习率
lr = 3e-5
# 取多少训练集的数据作为验证集
validation_ratio = 0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 每多少步，打印一次loss
log_per_step = 50

# 数据集所在位置
dataset_dir = Path("/root/autodl-tmp/gitdir/ai_summer_camp/基于论文摘要的文本分类与关键词抽取挑战赛公开数据")
os.makedirs(dataset_dir) if not os.path.exists(dataset_dir) else ''

# 模型存储路径
model_dir = Path("/root/autodl-tmp/gitdir/ai_summer_camp/model/bert_checkpoints")
# 如果模型目录不存在，则创建一个
os.makedirs(model_dir) if not os.path.exists(model_dir) else ''

print("Device:", device)


Device: cuda


##### 数据收集与准备

> 在赛题主页下载数据，读取数据集，数据预处理（考虑数据扩增）

In [3]:
# 读取数据集，进行数据处理

pd_train_data = pd.read_csv('/root/autodl-tmp/gitdir/ai_summer_camp/基于论文摘要的文本分类与关键词抽取挑战赛公开数据/train.csv')
pd_train_data['title'] = pd_train_data['title'].fillna('')
pd_train_data['abstract'] = pd_train_data['abstract'].fillna('')

test_data = pd.read_csv('/root/autodl-tmp/gitdir/ai_summer_camp/基于论文摘要的文本分类与关键词抽取挑战赛测试集B/testB.csv')
test_data['title'] = test_data['title'].fillna('')
test_data['abstract'] = test_data['abstract'].fillna('')
pd_train_data['text'] = pd_train_data['title'].fillna('') + ' ' +  pd_train_data['author'].fillna('') + ' ' + pd_train_data['abstract'].fillna('')+ ' ' + pd_train_data['Keywords'].fillna('')
test_data['text'] = test_data['title'].fillna('') + ' ' +  test_data['author'].fillna('') + ' ' + test_data['abstract'].fillna('')+ ' ' + pd_train_data['Keywords'].fillna('')
#test['Keywords'] = test['title'].fillna('')

test_data['title'].head(5)

0    Tobacco Consumption and High-Sensitivity Cardi...
1    Approaching towards sustainable supply chain u...
2    Does globalization matter for ecological footp...
3    Myths and Misconceptions About University Stud...
4    Antioxidant Status of Rat Liver Mitochondria u...
Name: title, dtype: object

In [4]:
# 从训练集中随机采样测试集
validation_data = pd_train_data.sample(frac=validation_ratio)
train_data = pd_train_data[~pd_train_data.index.isin(validation_data.index)]


In [5]:
#查看训练数据集的列名
print(pd_train_data.columns.values)
#查看测试数据集的列名
print(test_data.columns.values)


['uuid' 'title' 'author' 'abstract' 'Keywords' 'label' 'text']
['uuid' 'title' 'author' 'abstract' 'text']


##### 构建训练所需的dataloader与dataset

###### 定义dataset

In [6]:
# 构建Dataset
class MyDataset(Dataset):

    def __init__(self, mode='train'):
        super(MyDataset, self).__init__()
        self.mode = mode
        # 拿到对应的数据
        if mode == 'train':
            self.dataset = train_data
        elif mode == 'validation':
            self.dataset = validation_data
        elif mode == 'test':
            # 如果是测试模式，则返回内容和uuid。拿uuid做target主要是方便后面写入结果。
            self.dataset = test_data
        else:
            raise Exception("Unknown mode {}".format(mode))

    def __getitem__(self, index):
        # 取第index条
        data = self.dataset.iloc[index]
        # 取其内容
        text = data['text']
        # 根据状态返回内容
        if self.mode == 'test':
            # 如果是test，将uuid做为target
            label = data['uuid']
        else:
            label = data['label']
        # 返回内容和label
        return text, label

    def __len__(self):
        return len(self.dataset)


In [7]:
train_dataset = MyDataset('train')
validation_dataset = MyDataset('validation')


In [8]:
train_dataset.__getitem__(0)


('Accessible Visual Artworks for Blind and Visually Impaired People: Comparing a Multimodal Approach with Tactile Graphics Quero, Luis Cavazos; Bartolome, Jorge Iranzo; Cho, Jundong Despite the use of tactile graphics and audio guides, blind and visually impaired people still face challenges to experience and understand visual artworks independently at art exhibitions. Art museums and other art places are increasingly exploring the use of interactive guides to make their collections more accessible. In this work, we describe our approach to an interactive multimodal guide prototype that uses audio and tactile modalities to improve the autonomous access to information and experience of visual artworks. The prototype is composed of a touch-sensitive 2.5D artwork relief model that can be freely explored by touch. Users can access localized verbal descriptions and audio by performing touch gestures on the surface while listening to themed background music along. We present the design requi

###### 构造Dataloader

In [9]:
#获取Bert预训练模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [10]:
#接着构造我们的Dataloader。
#我们需要定义一下collate_fn，在其中完成对句子进行编码、填充、组装batch等动作：
def collate_fn(batch):
    """
    将一个batch的文本句子转成tensor，并组成batch。
    :param batch: 一个batch的句子，例如: [('推文', target), ('推文', target), ...]
    :return: 处理后的结果，例如：
             src: {'input_ids': tensor([[ 101, ..., 102, 0, 0, ...], ...]), 'attention_mask': tensor([[1, ..., 1, 0, ...], ...])}
             target：[1, 1, 0, ...]
    """
    text, label = zip(*batch)
    text, label = list(text), list(label)

    # src是要送给bert的，所以不需要特殊处理，直接用tokenizer的结果即可
    # padding='max_length' 不够长度的进行填充
    # truncation=True 长度过长的进行裁剪
    src = tokenizer(text, padding='max_length', max_length=text_max_length, return_tensors='pt', truncation=True)

    return src, torch.LongTensor(label)


In [11]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


In [12]:
# 里面iter 是迭代器，next是输出迭代器的下一个值

inputs, targets = next(iter(train_loader))
print("inputs:", inputs)
print("targets:", targets)


inputs: {'input_ids': tensor([[  101,  7605,  4874,  ..., 10624,  6914,   102],
        [  101,  2309,  1011,  ...,  5679,  1999,   102],
        [  101, 27218,  1011,  ..., 11004,  4609,   102],
        ...,
        [  101,  7987,  3593,  ...,  2122,  7876,   102],
        [  101,  2156,  2129,  ...,  6454,  1010,   102],
        [  101, 21598, 16913,  ...,  1058, 18939,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}
targets: tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])


##### 定义模型

In [13]:
#定义预测模型，该模型由bert模型加上最后的预测层组成
class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()

        # 加载bert模型
        self.bert = BertModel.from_pretrained('bert-base-uncased', mirror='tuna')

        # 最后的预测层
        self.predictor = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, src):
        """
        :param src: 分词后的推文数据
        """

        # 将src直接序列解包传入bert，因为bert和tokenizer是一套的，所以可以这么做。
        # 得到encoder的输出，用最前面[CLS]的输出作为最终线性层的输入
        outputs = self.bert(**src).last_hidden_state[:, 0, :]

        # 使用线性层来做最终的预测
        return self.predictor(outputs)


In [14]:
model = MyModel()
model = model.to(device)


##### 定义出损失函数和优化器

In [15]:
#定义出损失函数和优化器。这里使用Binary Cross Entropy：
criteria = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [16]:
# 由于inputs是字典类型的，定义一个辅助函数帮助to(device)
# Python 字典(Dictionary) items() 函数以列表返回可遍历的(键, 值) 元组数组。
def to_device(dict_tensors):
    result_tensors = {}
    for key, value in dict_tensors.items():
        result_tensors[key] = value.to(device)
    return result_tensors


##### 定义验证方法

In [17]:
#定义一个验证方法，获取到验证集的精准率和loss
def validate():
    model.eval()
    total_loss = 0.
    total_correct = 0
    for inputs, targets in validation_loader:
        inputs, targets = to_device(inputs), targets.to(device)
        outputs = model(inputs)
        loss = criteria(outputs.view(-1), targets.float())
        total_loss += float(loss)

        correct_num = (((outputs >= 0.5).float() * 1).flatten() == targets).sum()
        total_correct += correct_num

    return total_correct / len(validation_dataset), total_loss / len(validation_dataset)


##### 模型训练、评估

In [18]:
# 首先将模型调成训练模式
model.train()

# 清空一下cuda缓存
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 定义几个变量，帮助打印loss
total_loss = 0.
# 记录步数
step = 0

# 记录在验证集上最好的准确率
best_accuracy = 0

# 开始训练
for epoch in range(epochs):
    model.train()
    for i, (inputs, targets) in enumerate(train_loader):
        # 从batch中拿到训练数据
        inputs, targets = to_device(inputs), targets.to(device)
        # 传入模型进行前向传递
        outputs = model(inputs)
        # 计算损失
        loss = criteria(outputs.view(-1), targets.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += float(loss)
        step += 1

        if step % log_per_step == 0:
            print("Epoch {}/{}, Step: {}/{}, total loss:{:.4f}".format(epoch+1, epochs, i, len(train_loader), total_loss))
            total_loss = 0

        del inputs, targets

    # 一个epoch后，使用过验证集进行验证
    accuracy, validation_loss = validate()
    print("Epoch {}, accuracy: {:.4f}, validation loss: {:.4f}".format(epoch+1, accuracy, validation_loss))
    torch.save(model, model_dir / f"model_{epoch}.pt")

    # 保存最好的模型
    if accuracy > best_accuracy:
        torch.save(model, model_dir / f"model_best.pt")
        best_accuracy = accuracy


Epoch 1/100, Step: 49/338, total loss:16.5111
Epoch 1/100, Step: 99/338, total loss:6.5136
Epoch 1/100, Step: 149/338, total loss:5.6502
Epoch 1/100, Step: 199/338, total loss:6.6170
Epoch 1/100, Step: 249/338, total loss:5.2193
Epoch 1/100, Step: 299/338, total loss:3.8585
Epoch 1, accuracy: 0.9617, validation loss: 0.0078
Epoch 2/100, Step: 11/338, total loss:6.6585
Epoch 2/100, Step: 61/338, total loss:3.4986
Epoch 2/100, Step: 111/338, total loss:3.1230
Epoch 2/100, Step: 161/338, total loss:4.4627
Epoch 2/100, Step: 211/338, total loss:3.9473
Epoch 2/100, Step: 261/338, total loss:4.6345
Epoch 2/100, Step: 311/338, total loss:3.9789
Epoch 2, accuracy: 0.9583, validation loss: 0.0063
Epoch 3/100, Step: 23/338, total loss:2.7229
Epoch 3/100, Step: 73/338, total loss:2.8375
Epoch 3/100, Step: 123/338, total loss:2.6576
Epoch 3/100, Step: 173/338, total loss:2.6944
Epoch 3/100, Step: 223/338, total loss:2.0251
Epoch 3/100, Step: 273/338, total loss:2.9500
Epoch 3/100, Step: 323/338, t

In [19]:
#加载最好的模型，然后进行测试集的预测
model = torch.load(model_dir / f"model_best.pt")
model = model.eval()


In [20]:
test_dataset = MyDataset('test')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


##### 结果输出

###### 提交需要符合提交样例结果

In [21]:
results = []
for inputs, ids in test_loader:
    outputs = model(inputs.to(device))
    outputs = (outputs >= 0.5).int().flatten().tolist()
    ids = ids.tolist()
    results = results + [(id, result) for result, id in zip(outputs, ids)]


In [22]:
test_label = [pair[1] for pair in results]
test_data['label'] = test_label
test_data['Keywords'] = test_data['title'].fillna('')
test_data[['uuid', 'Keywords', 'label']].to_csv('submit_task2.csv', index=None)